Since I'm going to reference a single dataframe a lot, it makes sense to create it once instead of constantly going to the postegresql server for data retrieval

In [1]:
import sys
sys.path.insert(0, '../..')

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import helpers.laxdb as laxdb
LaxDB = laxdb.LaxDB

SELECT data from Postegresql server

In [2]:
box = """
SELECT * from ncaa.box_scores;
"""
boxdata = LaxDB().query(box)

teams = """
SELECT id, name from ncaa.teams;
"""
teamdata = LaxDB().query(teams)

games = """
SELECT id, season_id from ncaa.games;
"""
gamedata = LaxDB().query(games)

seasons = """
SELECT id, year from ncaa.seasons;
"""
seasondata = LaxDB().query(seasons)

No need to leave the connection open after the data is retrieved

In [3]:
LaxDB().close()

In [16]:
seasondata.head()

,id,year
0,2,2016
1,1,2015
2,4,2018
3,5,2019
4,6,2020


In [4]:
gamedata.rename(columns={'id':'gameid'}, inplace=True)
gamedata.head()

,gameid,season_id
0,55,1
1,57,1
2,64,1
3,65,1
4,66,1


merge seasons with game ids

In [5]:
game_df = seasondata.merge(gamedata, left_on='id', right_on='season_id')
game_df.drop('id', axis=1, inplace=True)
game_df.head()

,year,gameid,season_id
0,2016,533,2
1,2016,535,2
2,2016,539,2
3,2016,545,2
4,2016,598,2


Merge the two sets of data on common field

In [6]:
boxscores_df = teamdata.merge(boxdata, left_on='id', right_on='team_id')
boxscores_df.head()

,id_x,name,id_y,game_id,team_id,player_id,position,player_name,goals,assists,...,caused_turnovers,faceoffs_won,faceoffs_taken,penalties,penalty_time,goalie_minutes,goals_allowed,goalie_saves,created_at,updated_at
0,2,Binghamton,4712,1,2,41.0,D,Chris Bechle,0,0,...,0,0,0,1,30,0,0,0,2023-10-24 16:37:17.319077,2023-10-24 16:37:17.319077
1,2,Binghamton,4713,1,2,48.0,D,George Diegnan,0,0,...,0,1,2,0,0,0,0,0,2023-10-24 16:37:17.319077,2023-10-24 16:37:17.319077
2,2,Binghamton,4714,1,2,51.0,D,Sean Finnigan,0,0,...,0,0,0,0,0,0,0,0,2023-10-24 16:37:17.319077,2023-10-24 16:37:17.319077
3,2,Binghamton,4715,1,2,57.0,M,Matt Kaser,0,1,...,0,0,0,0,0,0,0,0,2023-10-24 16:37:17.319077,2023-10-24 16:37:17.319077
4,2,Binghamton,4716,1,2,63.0,M,Anthony Lombardo,0,0,...,0,0,0,0,0,0,0,0,2023-10-24 16:37:17.319077,2023-10-24 16:37:17.319077


Change column name 'goalie_minutes' to 'goalie_seconds'

In [8]:
boxscores_df.rename(columns={'goalie_minutes':'goalie_seconds'}, inplace=True)

Merge boxscores with game_df to add the year of the games

In [9]:
cluster_df = game_df.merge(boxscores_df, left_on='gameid', right_on='game_id')
cluster_df.head()

,year,gameid,season_id,id_x,name,id_y,game_id,team_id,player_id,position,...,caused_turnovers,faceoffs_won,faceoffs_taken,penalties,penalty_time,goalie_seconds,goals_allowed,goalie_saves,created_at,updated_at
0,2016,533,2,70,Brown,34868,533,70,3065.0,A,...,0,0,0,0,0,0,0,0,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633
1,2016,533,2,70,Brown,34869,533,70,3066.0,A,...,0,0,0,0,0,0,0,0,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633
2,2016,533,2,70,Brown,34870,533,70,3074.0,M,...,1,19,27,0,0,0,0,0,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633
3,2016,533,2,70,Brown,34871,533,70,3077.0,G,...,0,0,0,0,0,3600,12,11,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633
4,2016,533,2,70,Brown,34872,533,70,3078.0,M,...,1,0,0,0,0,0,0,0,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633


check games by year

In [10]:
cluster_df.groupby('year')['gameid'].count()

year
2015    130842
2016    136893
2017    140715
2018    146270
2019    151328
2020     52822
2021     97522
2022    159183
2023    164715
Name: gameid, dtype: int64

What are all of the unique positions in this dataframe

In [28]:
cluster_df['position'].unique()

array(['A', 'M', 'G', 'D', '', 'LSM', 'GK', 'FO', 'MF', 'MFO', 'DM', 'AM',
       'LS', 'F', '6', 'S', 'O', 'AS', 'SSDM', None, 'FOGO', 'MA', 'FA',
       'L', 'DEF', 'ATT', 'MID', 'FOS', 'MD', 'FOM', 'DL', 'SR', 'FS',
       'CD', 'N', 'DK', 'SS', 'ATK', '3', 'GOAL', 'DLS', 'AQ', 'AT',
       'PCS', 'C', 'MK', 'DG', 'K', 'ID', 'LM', 'LMS', '8', '45', 'B',
       '4', 'GF', 'MIC', 'FK', 'GT', '1', 'DDM', 'DLSM', 'DMF', 'F0',
       'SLM', '5', 'D08', 'MB', 'DST', 'DFO', 'AMF', 'IH', 'Q', 'SSD',
       '2', 'GO', 'ATMD', 'CK', 'MIDF', 'MIG', 'ATM', 'NF', '31', 'SSM',
       'FSO', 'AFO', 'MDM', 'LSMD', '16', 'NA', '0', 'AD', 'LP', 'W',
       'FM', 'MG', 'DLMS', 'LSMF', 'MFA', 'LPM', 'LDM', '35', 'RW', 'LSD',
       'DB', 'FW', '9', 'AA', 'FOR', '27'], dtype=object)

That's interesting. There are letters and letter groups that make sense, but then there are numbers. Let's take a look at 27

In [7]:
twensev = np.where(cluster_df['position'] == '27')
twensev

(array([793184], dtype=int64),)

This should show up as an outlier

In [8]:
cluster_df.iloc[793184]

id_x                                      3318
name                          Westminster (UT)
id_y                                   1222376
game_id                                  26505
team_id                                   3318
player_id                             124438.0
position                                    27
player_name                        Jacob Parks
goals                                        0
assists                                      0
points                                       0
shots                                        2
shots_on_goal                                0
man_up_goals                                 0
man_down_goals                               0
ground_balls                                 1
turnovers                                    1
caused_turnovers                             0
faceoffs_won                                 0
faceoffs_taken                               0
penalties                                    0
penalty_time 

Save the cluster dataframe to csv

In [11]:
cluster_df.to_csv('cluster_df')

Load to the csv to confirm it's the same

In [12]:
df = pd.read_csv('cluster_df')
df.head()

,Unnamed: 0,year,gameid,season_id,id_x,name,id_y,game_id,team_id,player_id,...,caused_turnovers,faceoffs_won,faceoffs_taken,penalties,penalty_time,goalie_seconds,goals_allowed,goalie_saves,created_at,updated_at
0,0,2016,533,2,70,Brown,34868,533,70,3065.0,...,0,0,0,0,0,0,0,0,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633
1,1,2016,533,2,70,Brown,34869,533,70,3066.0,...,0,0,0,0,0,0,0,0,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633
2,2,2016,533,2,70,Brown,34870,533,70,3074.0,...,1,19,27,0,0,0,0,0,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633
3,3,2016,533,2,70,Brown,34871,533,70,3077.0,...,0,0,0,0,0,3600,12,11,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633
4,4,2016,533,2,70,Brown,34872,533,70,3078.0,...,1,0,0,0,0,0,0,0,2023-10-27 12:25:20.222633,2023-10-27 12:25:20.222633


In [31]:
df.columns

Index(['Unnamed: 0', 'year', 'gameid', 'season_id', 'id_x', 'name', 'id_y',
       'game_id', 'team_id', 'player_id', 'position', 'player_name', 'goals',
       'assists', 'points', 'shots', 'shots_on_goal', 'man_up_goals',
       'man_down_goals', 'ground_balls', 'turnovers', 'caused_turnovers',
       'faceoffs_won', 'faceoffs_taken', 'penalties', 'penalty_time',
       'goalie_seconds', 'goals_allowed', 'goalie_saves', 'created_at',
       'updated_at'],
      dtype='object')

All looks good!!